# Training PoC

In [1]:
import sys

sys.path.append("..")
sys.path.append("../../inputs")

In [2]:
import logging

In [3]:
logging.basicConfig(
    # filename=__file__.replace('.py', '.log'),
    level=logging.getLevelName("INFO"),
    format="%(asctime)s [%(levelname)s] [%(module)s] %(message)s",
)

In [4]:
log = logging.getLogger(__name__)

In [5]:
from omegaconf import OmegaConf

In [6]:
c = OmegaConf.load("../config/main.yaml")

In [7]:
c.settings.debug = True
c.wandb.enabled = False
c.settings.dirs.working = ".."
c.settings.dirs.input = "../../inputs/"

In [8]:
log.info(OmegaConf.to_yaml(c))

2022-01-25 21:26:23,233 [INFO] [3244290467] defaults:
- _self_
hydra:
  run:
    dir: ../outputs/${now:%Y-%m-%d_%H-%M-%S}
  job_logging:
    formatters:
      simple:
        format: '%(asctime)s [%(levelname)s][%(module)s] %(message)s'
wandb:
  enabled: false
  entity: imokuri
  project: ump
  dir: ${hydra:runtime.cwd}/../cache
settings:
  print_freq: 100
  gpus: 6,7
  dirs:
    working: ..
    input: ../../inputs/
  inputs:
  - train.csv
  - example_test.csv
  - example_sample_submission.csv
  debug: true
  n_debug_data: 100000
  amp: true
  multi_gpu: true
params:
  seed: 440
  n_class: 1
  n_fold: 7
  skip_training: false
  epoch: 10
  es_patience: 0
  batch_size: 512
  gradient_acc_step: 1
  max_grad_norm: 1000
  fold: group
  group_name: investment_id
  label_name: target
  dataset: ump_1
  model: ump_1
  pretrained: []
  criterion: RMSELoss
  optimizer: Adam
  scheduler: CosineAnnealingWarmupRestarts
  lr: 0.001
  min_lr: 1.0e-06
  weight_decay: 1.0e-05
  label_smoothing: 1.0e-0

## Main

In [9]:
import os
import time

import pandas as pd
import torch.cuda.amp as amp

In [10]:
import src.utils as utils
from src.load_data import InputData
from src.make_dataset import make_dataset
from src.make_model import make_model
from src.make_loss import make_criterion, make_optimizer, make_scheduler
from src.run_loop import EarlyStopping
from src.time_series_api import TimeSeriesAPI
from src.utils import AverageMeter

In [11]:
utils.debug_settings(c)

2022-01-25 21:26:24,639 [INFO] [utils] Enable debug mode.


In [12]:
run = utils.setup_wandb(c)

In [13]:
log.info(f"Started at {os.path.basename(os.getcwd())}")

2022-01-25 21:26:24,647 [INFO] [518690292] Started at notebooks


In [14]:
utils.fix_seed(c.params.seed)

2022-01-25 21:26:24,650 [INFO] [utils] Fix seed: 440


In [15]:
device = utils.gpu_settings(c)

2022-01-25 21:26:24,655 [INFO] [utils] CUDA_VISIBLE_DEVICES: 6,7
2022-01-25 21:26:24,701 [INFO] [utils] torch device: cuda, device count: 2


In [16]:
input = InputData(c)

2022-01-25 21:26:24,705 [INFO] [load_data] Load feather file. path: ../../inputs/train.f
2022-01-25 21:26:43,907 [INFO] [utils] Mem. usage decreased to 365Mb: 49% reduction
2022-01-25 21:26:43,943 [INFO] [load_data] Load feather file. path: ../../inputs/example_test.f
2022-01-25 21:26:44,032 [INFO] [utils] Mem. usage decreased to 0.0Mb: 49% reduction
2022-01-25 21:26:44,033 [INFO] [load_data] Load feather file. path: ../../inputs/example_sample_submission.f
2022-01-25 21:26:44,037 [INFO] [utils] Mem. usage decreased to 0.0Mb: 34% reduction


In [17]:
input.train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3141410 entries, 0 to 3141409
Columns: 305 entries, row_id to fold
dtypes: float32(302), int16(2), object(1)
memory usage: 3.6+ GB


In [18]:
oof_df = pd.DataFrame()
losses = utils.AverageMeter()

In [19]:
for fold in range(c.params.n_fold):
    log.info(f"========== fold {fold} training ==========")
    utils.fix_seed(c.params.seed + fold)

    ####################################################
    # _oof_df, score, loss = train_fold(c, input.train, fold, device)
    df = input.train
    
    trn_idx = df[df["fold"] != fold].index                                                          
    val_idx = df[df["fold"] == fold].index                                                          
    log.info( f"Num of training data: {len(trn_idx)}, num of validation data: {len(val_idx)}")          
    
    train_folds = df.loc[trn_idx].reset_index(drop=True)  
    valid_folds = df.loc[val_idx].reset_index(drop=True)  

    train_ds = make_dataset(c, train_folds)

    model = make_model(c, device)                      

    criterion = make_criterion(c)                      
    optimizer = make_optimizer(c, model)               
    scaler = amp.GradScaler(enabled=c.settings.amp)    
    scheduler = make_scheduler(c, optimizer, train_ds) 

    es = EarlyStopping(c=c, fold=fold)                     

    for epoch in range(c.params.epoch):
        start_time = time.time()
    
        iter_train = TimeSeriesAPI(train_folds)
        avg_train_loss = AverageMeter()  
        
        for train_df, train_pred_df in iter_train:
            print(len(train_df))
            
            iter_train.predict(train_pred_df)
            
        break
        
    
    
    break
    
    
    ####################################################
    
    oof_df = pd.concat([oof_df, _oof_df])
    losses.update(loss)

    log.info(f"========== fold {fold} result ==========")
    record_result(c, _oof_df, fold, loss)

    if c.settings.debug:
        break

2022-01-25 21:26:44,057 [INFO] [1237397599] ========== fold 0 training ==========
2022-01-25 21:26:44,058 [INFO] [utils] Fix seed: 440
2022-01-25 21:26:45,405 [INFO] [1237397599] Num of training data: 2094274, num of validation data: 1047136


1516
1524
1523
1530
1516
1525
1525
1515
1508
1516
1517
1523
1518
1512
1513
1514
1506
1532
1535
1524
1536
1509
1503
1516
1517
1527
1511
1499
1529
1533
1528
1550
1515
1472
1522
1526
1527
1517
1538
1542
1526
1540
1517
1544
1539
1533
1541
1527


/opt/miniconda3/envs/py37-all-in-one/lib/python3.7/site-packages/scipy/stats/stats.py:4023: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(PearsonRConstantInputWarning())


1539
1533
1531
1538
1516
1531
1535
1524
1506
1514
1521
1516
1528
1529
1522
1511
1521
1521
1506
1516
1518
1520
1517
1515
1516
1523
1513
1511
1449
1498
1508
1500
1502
1504
1493
1498
1473
1499
1505
1511
1509
1511
1512
1504
1511
1505
1507
1511
1498
1503
1499
1499
1499
1490
1504
1497
1497
1498
1491
1477
1491
1504
1498
1486
1505
1489
1496
1496
1477
1481
1471
1486
1487
1481
1492
1494
1487
1495
1479
1482
1472
1479
1481
1492
1492
1479
1479
1489
1492
1494
1487
1483
1478
1494
1499
1487
1491
1499
1504
1508
1493
1500
1492
1494
1486
1483
1481
1486
1495
1493
1487
1492
1487
1491
1508
1495
1498
1490
1498
1500
1493
1491
1484
1487
1476
1491
1482
1495
1484
1490
1486
1486
1487
1502
1495
1509
1487
1494
1503
1495
1498
1494
1507
1502
1507
1514
1517
1508
1510
1505
1496
1491
1510
1515
1500
1505
1518
1521
1532
1508
1513
1510
1516
1513
1510
1512
1513
1507
1498
1488
1481
1491
1466
1472
1476
1482
1481
1459
1470
1453
1360
1443
1479
1496
1495
1505
1513
1500
1491
1267
1482
1494
1496
1500
1489
1501
1508
1480
1504
1517
